##### This notebook will be used to `convert the video data (landmarks) into numpy files`. We also include `data augmentation` in the process via `horizontal flipping` (left/right handed signers) and `zooming-in`. The steps are as follows:

1. `Install` Dependencies
2. `Call functions` for:   
`Drawing/Extracting` landmarks using `MP Holistic`,  
`Zooming in` on the image for `data augmentation`,  
`Plotting` the images for `visualization`.
3. `Setup Folders` for Collection
4. `Collect Landmarks` per frame for Training and Testing. `Store` them as numpy files in DATA_PATH, FLIPPED_DATA_PATH and ZOOMED_DATA_PATH.
5. `Converge DATA_PATH folders into MP_Data folder.`  

1. `Install` Dependencies

In [13]:
# %pip install tensorflow==2.6 opencv-python mediapipe==0.9.1.0 scikit-learn matplotlib
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import os
import mediapipe as mp

2. Call functions for:   
`Drawing/Extracting` landmarks using `MP Holistic`,  
`Zooming in` on the image for `data augmentation`,  
`Plotting` the images for `visualization`.

In [14]:
# Keypoints Portion
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
   #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
    
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )
    
#Extracting Keypoints into a numpy array
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4) 
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh]) # single row of data for each image

In [15]:
# Function for zooming image by 1.2x
def zoom_frame(image, zoom=1.2, angle=0, coord=None):
    
    cy, cx = [ i/2 for i in image.shape[:-1] ] if coord is None else coord[::-1] # Centered at "coord", if given, else the image center.
    
    rot_mat = cv2.getRotationMatrix2D((cx,cy), angle, zoom) # Rotation matrix.
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR) 
    
    return result

# Plot the images
def plot_images(image, flipped_image, zoomed_image):
    # Plot the first image
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title('Original Image')

    # Plot the second image
    plt.subplot(1, 3, 2)
    plt.imshow(flipped_image)
    plt.title('H. Flipped Image')

    # Plot the third image
    plt.subplot(1, 3, 3)
    plt.imshow(zoomed_image)
    plt.title('Zoomed Image')

    # Show the plot
    plt.show()

3. `Setup Folders` for Collection

In [16]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('DATA') 
FLIPPED_DATA_PATH = os.path.join('FLIPPED_DATA')
ZOOMED_DATA_PATH = os.path.join('ZOOMED_DATA')

# Actions that we try to detect
actions = np.array(['ako',  'bakit', 'F', 'hi', 'hindi', 'ikaw',  'kamusta', 'L', 'maganda', 'magandang umaga', 'N', 'O', 'oo', 'P', 'salamat'])

# number of videos we want to use for training
no_sequences = 14

# Videos are going to be 30 frames in length
sequence_length = 30

In [17]:
# Define input directories
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            os.makedirs(os.path.join(FLIPPED_DATA_PATH, action, str(sequence)))
            os.makedirs(os.path.join(ZOOMED_DATA_PATH, action, str(sequence)))
        except:
            pass

4. `Collect Landmarks` for Training and Testing. `Store` them as numpy files in DATA_PATH.

In [18]:
# Set up Mediapipe Holistic model
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

input_dir = '../videos_copy/signed_videos_processed'

# Loop through actions
for action in actions:
    # Loop through sequences
    filename_action = os.path.join(input_dir, action)
    # no_sequences = the number of files in filename_action
    # no_sequences = len([name for name in os.listdir(filename_action) if os.path.isfile(os.path.join(filename_action, name))])
    for sequence in range(no_sequences):       
        filename = os.path.join(input_dir, action, '{}.mp4'.format(sequence))
        cap = cv2.VideoCapture(filename) # Open video file
        # Loop through sequence length
        for frame_num in range(sequence_length):
            # Loop through video frames
            # Read video frame
            ret, frame = cap.read()
            zoomed_frame = zoom_frame(frame)
            flipped_frame = cv2.flip(frame, 1)

            if not ret:
                # End of video
                break

            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            zoomed_image, zoomed_results = mediapipe_detection(zoomed_frame, holistic)
            flipped_image, flipped_results = mediapipe_detection(flipped_frame, holistic)
            
            # Draw landmarks
            draw_landmarks(image, results)
            draw_landmarks(flipped_image, flipped_results)
            draw_landmarks(zoomed_image, zoomed_results)
 
            # NEW Export keypoints
            keypoints = extract_keypoints(results)
            flipped_keypoints = extract_keypoints(flipped_results)
            zoomed_keypoints = extract_keypoints(zoomed_results) 
           
            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
            flipped_npy_path = os.path.join(FLIPPED_DATA_PATH, action, str(sequence), str(frame_num))
            zoomed_npy_path = os.path.join(ZOOMED_DATA_PATH, action, str(sequence), str(frame_num))
            
            np.save(npy_path, keypoints)
            np.save(flipped_npy_path, flipped_keypoints)
            np.save(zoomed_npy_path, zoomed_keypoints)
            
            # Optional: Plot Images
            # plot_images(image, flipped_image, zoomed_image)
                
        # Release video file
        cap.release()
cv2.destroyAllWindows()

[ WARN:0@4738.955] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@4738.955] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin714 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@4738.955] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@4738.955] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules

In [ ]:
cap.release()
cv2.destroyAllWindows()

5. `Converge DATA_PATH folders into One folder.`  
`Move all the DATA_PATH folders` into a single folder called `MP_Data`. This is done so that the model can be trained on all the data at once.

In [20]:
import os
import shutil

# folders = [FLIPPED_DATA_PATH, ZOOMED_DATA_PATH]
# new_folder = DATA_PATH

folders = ['MP_Data_Old', 'MP_Data_Gallen'] # Used for convenience. Use the commented above code for the actual folders.
new_folder = 'MP_Data'

if not os.path.exists(new_folder):
    os.makedirs(new_folder)

for folder in folders:
    subdirectories = os.listdir(folder)
    for subdirectory in subdirectories:
        if subdirectory == '.DS_Store':
          continue
        subsubdirectories = os.listdir(os.path.join(folder, subdirectory))
        for subsubdirectory in subsubdirectories:
            src_path = os.path.join(folder, subdirectory, subsubdirectory)
            dst_path = os.path.join(new_folder, subdirectory, subsubdirectory)
            if os.path.exists(dst_path):
                existing_subsubdirectories = [int(name) for name in os.listdir(os.path.join(new_folder, subdirectory))]
                new_subsubdirectory = str(max(existing_subsubdirectories) + 1)
                dst_path = os.path.join(new_folder, subdirectory, new_subsubdirectory)
            if not os.path.exists(dst_path):
                os.makedirs(dst_path)
            for file in os.listdir(src_path):
                shutil.copy2(os.path.join(src_path, file), dst_path)